# Chapter 5 applied exercises

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

## 5

In chapter 4 we used logistic regression to predict the probability of *default* using *income* and *balance* on the *Default* data set. We will now estimate the test error of this logistic regression model using the validation set approach.

a) Fit a logistic regression model that uses *income* and *balance* to predict *default*

b) Using the validation set approach, estimate the test error of this mode.

c) Repeat the process in b) 3 times, using 3 different splits of the observations into a training set and a test set. Comment on the results obtained.

d) Now consider a logistic regression model that predicts the probability of *default* using *income*, *balance*, and a dummy variable for *student*. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for *student* leads to a reduction in the test error rate.

In [ ]:
df = sm.datasets.get_rdataset("Default", "ISLR", cache=True).data.pipe(pd.get_dummies, columns=["default", "student"], drop_first=True)

In [ ]:
df.columns

In [ ]:
y = df["default_Yes"]
X = sm.add_constant(df[["income", "balance"]])

In [ ]:
def train_test_validation_error():
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    logit = sm.Logit(y_train, X_train).fit()
    predict_prob = logit.predict(X_test)
    predict_class = pd.Series(data=0, index=predict_prob.index)
    predict_class.loc[predict_prob > 0.5] = 1
    validation_error = (predict_class.values != y_test.values).mean()
    return validation_error

In [ ]:
validation_errors = [train_test_validation_error() for _ in range(3)]

In [ ]:
validation_errors

Not really sure what to comment. They're all quite similar, which is good. If I did this a few more times I could make some distributional assumptions about the error rate. Note that this is a really imbalanced class, so my low error rate isn't that impressive

d) Now consider a logistic regression model that predicts the probability of *default* using *income*, *balance* and a dummy variable for *student*. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for *student* leads to a reduction in the test error rate

In [ ]:
X = sm.add_constant(df[["income", "balance", "student_Yes"]])

In [ ]:
validation_errors = [train_test_validation_error() for _ in range(3)]

In [ ]:
validation_errors

Inconclusive result from this small test. One result is lower than obtained without the dummy, one is higher, and one is the same.

## 6

We continue to consider the use of a logistic regression model to predict the probability of *default* using *income* and *balance* on the *Default* data set. In particular, we will now compute estimates for the standard errors of the *income* and *balance* logistic regression coefficients in two different ways: 1) using the bootstrap, and 2) using the standard formula for computing the standard errors. 

a) Use statsmodels to determine the estimated standard errors for the coefficients associated with *income* and *balance* in the multiple logistic regression model

b) Write a function ```boot_fn```, that takes as input the *Default* data set as well as an index of observations,  and that outputs the coefficient estimates for *income* and *balance* in the multiple logistic regression model.

c) Use ```boot_fn```  to estimate the standard errors of the logistic regression coefficients for *income* and *balance*

d) comment on the results